In [1]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences

In [5]:
checking = pd.read_csv("../data/Checking.csv", encoding='iso-8859-1')
checking.head()

,Date,nps,Comment
0,4/17/2023 20:23,9,PRECIO
1,4/17/2023 20:21,9,ES LA PRIMERA VEZ Y SI ME SIENTO CON ALGO DE I...
2,4/17/2023 20:17,9,Es muy rapido
3,4/17/2023 20:17,10,Bueno
4,4/17/2023 20:10,9,Prcticidad
